尝试闭卷默写

全流程共计约2h（在上数据科学课）

整体流程以及各部件逻辑全部正确

个别函数接口忘了

最终有2个bug自己没找出来，借助ai
1. x[index] = tmp 写成了 x = tmp
2. a.reshape(1, -1) 没赋值为什么不会报错？嗷嗷压根没测试单样本

优化了 numerate_gradient 的索引部分
for index in np.ndindex(x.shape):

In [ ]:
import numpy as np

In [ ]:
def sigmoid(a):
    return 1 / (1 + np.exp(-a))

def relu(a):
    return np.maximum(a, 0)

def softmax(a):
    if a.ndim == 1:
        a = a.reshape(1, -1)
    a_max = np.max(a, axis=1, keepdims=True)
    return np.exp(a - a_max) / np.sum(np.exp(a - a_max), axis=1, keepdims=True)

def numerate_gradient(x, f):
    grad = np.zeros_like(x)

    for index in np.ndindex(x.shape):
        h = 1e-6
        tmp = x[index]
        x[index] = tmp + h
        fh1 = f(x)
        x[index] = tmp - h
        fh2 = f(x)
        x[index] = tmp
        grad[index] = (fh1 - fh2) / (2 * h)
    
    return grad

In [ ]:
class Net:
    def __init__(self,input_size=28*28, layer1_size=10, output_size=10) -> None:
        self.key = ['W1', 'b1', 'W2', 'b2']
        self.weight = {}
        self.weight['W1'] = np.random.randn(input_size, layer1_size)
        self.weight['b1'] = np.random.randn(layer1_size)
        self.weight['W2'] = np.random.randn(layer1_size, output_size)
        self.weight['b2'] = np.random.randn(output_size)
    
    def forward(self, x):
        a1 = np.dot(x, self.weight['W1']) + self.weight['b1']
        # z1 = sigmoid(a1)
        z1 = relu(a1)
        a2 = np.dot(z1, self.weight['W2']) + self.weight['b2']
        z2 = softmax(a2)
        return z2
    
    def loss(self, x, t):
        if x.ndim == 1:
            x = x.reshape(1, -1)
        
        y = self.forward(x)
        return -np.mean(np.log(y[np.arange(y.shape[0]), t]))
    
    def numeriate_gradient(self, x, t):
        grad = {}

        for k in self.key:
            f = lambda _:self.loss(x, t)  # 这里只是为了封装成 numerate_gradient 的接口 loss 内以隐含 weight 不用再传入
            grad[k] = numerate_gradient(self.weight[k], f)
        
        return grad

In [ ]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(
    root='../data',
    train=True,
    download=False,
    transform=transform
)

x_train_list = []
t_train_list = []
for image, label in train_dataset:
    x_train_list.append(image)
    t_train_list.append(label)

x_train = np.array(x_train_list)
x_train = x_train.reshape(x_train.shape[0], -1)
t_train = np.array(t_train_list)

In [ ]:
np.random.seed(42)

net = Net()

train_size = x_train.shape[0]
batch_size = 1
batch_mask = np.random.choice(train_size, batch_size)
max_iteration = 10
learning_rate = 0.01
loss_list = []

for i in range(max_iteration):
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    y_batch = net.forward(x_batch)
    grad = net.numeriate_gradient(x_batch, t_batch)
    for k in net.key:
        net.weight[k] -= learning_rate * grad[k]
    loss = net.loss(x_batch, t_batch)
    print(f"loss: {loss}")